In [1]:
import pandas as pd
import numpy as np
import json
import sys, os
import random
import numpy as np
import pandas as pd 

import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor,ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
data_dir = "../data/"

In [3]:
# train data
data = pd.read_csv(data_dir + "train_data.txt", sep="\t")
data["launch_seq"] = data.launch_seq.apply(lambda x: json.loads(x))
data["playtime_seq"] = data.playtime_seq.apply(lambda x: json.loads(x))
data["duration_prefer"] = data.duration_prefer.apply(lambda x: json.loads(x))
data["interact_prefer"] = data.interact_prefer.apply(lambda x: json.loads(x))

test_data = pd.read_csv(data_dir + "test_data.txt", sep="\t")
test_data["launch_seq"] = test_data.launch_seq.apply(lambda x: json.loads(x))
test_data["playtime_seq"] = test_data.playtime_seq.apply(lambda x: json.loads(x))
test_data["duration_prefer"] = test_data.duration_prefer.apply(lambda x: json.loads(x))
test_data["interact_prefer"] = test_data.interact_prefer.apply(lambda x: json.loads(x))
test_data['label'] = 0

#data = data.sample(frac=1).reset_index(drop=True)


In [ ]:
data.head()

In [4]:
class WSDMDataset(Dataset):
    def __init__(self, df):
        super(WSDMDataset, self).__init__()
        self.df = df
        self.feat_col = list(set(self.df.columns) - set(['user_id', 'end_date', 'label', 'launch_seq', 'playtime_seq', 
                'duration_prefer', 'interact_prefer']))
        self.df_feat = self.df[self.feat_col]
    
    def __getitem__(self, index):
        launch_seq = self.df['launch_seq'].iloc[index]
        playtime_seq = self.df['playtime_seq'].iloc[index]
        duration_prefer = self.df['duration_prefer'].iloc[index]
        interact_prefer = self.df['interact_prefer'].iloc[index]

        feat = self.df_feat.iloc[index].values.astype(np.float32)

        launch_seq = torch.tensor(launch_seq, dtype=torch.float)
        playtime_seq = torch.tensor(playtime_seq, dtype=torch.float)
        duration_prefer = torch.tensor(duration_prefer, dtype=torch.float)
        interact_prefer = torch.tensor(interact_prefer, dtype=torch.float)
        feat = torch.tensor(feat, dtype=torch.float)

        label = torch.tensor(self.df['label'].iloc[index], dtype=torch.float)
        return launch_seq, playtime_seq, duration_prefer, interact_prefer, feat, label

    def __len__(self):
        return len(self.df)

In [5]:
class WSDMDataModule(pl.LightningDataModule):
    def __init__(self, data):
        super().__init__()

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        self.train_dataset = WSDMDataset(data.iloc[:-6000])
        self.val_dataset = WSDMDataset(data.iloc[-6000:])
        self.test_dataset = None

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=CFG.batch_size, num_workers=16)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=CFG.batch_size*2, num_workers=16)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=CFG.batch_size*2, num_workers=16)

In [6]:
class WSDMModel(pl.LightningModule):
    def __init__(self):
        super(WSDMModel, self).__init__()
        
        self.launch_seq_gru = nn.GRU(1, 32)
        self.playtime_seq_gru = nn.GRU(1, 32)
        self.fc1 = nn.Linear(102, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, launch_seq, playtime_seq, duration_prefer, interact_prefer, feat):
        launch_seq = launch_seq.reshape((-1, 32, 1))
        playtime_seq = playtime_seq.reshape((-1, 32, 1))

        launch_seq_feat = self.launch_seq_gru(launch_seq)[0][:, :, 0]
        playtime_seq_feat = self.playtime_seq_gru(playtime_seq)[0][:, :, 0]
        
        all_feat = torch.cat([launch_seq_feat, playtime_seq_feat, duration_prefer, interact_prefer, feat], 1)
        
        all_feat_fc1 = self.fc1(all_feat)
        all_feat_fc2 = self.fc2(all_feat_fc1)

        return all_feat_fc2

In [7]:
class WSDMModule(pl.LightningModule):
    def __init__(self, features,learning_rate, weight_decay):
        super().__init__()
        self.learning_rate = learning_rate
        self.model = WSDMModel()

    def forward(self, features):
        return self.model(features,)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay = self.weight_decay)

    def training_step(self, batch, batch_idx):
        launch_seq, playtime_seq, duration_prefer, interact_prefer, feat, y = batch
        y_hat = self.model(launch_seq, playtime_seq, duration_prefer, interact_prefer, feat)
        loss = nn.MSELoss()(y_hat, y)
        return loss

    def validation_step(self, batch, batch_idx):
        launch_seq, playtime_seq, duration_prefer, interact_prefer, feat, y = batch
        y_hat = self.model(launch_seq, playtime_seq, duration_prefer, interact_prefer, feat)
        loss = nn.MSELoss()(y_hat, y)
        metrics = {'val_loss': loss}
        self.log_dict(metrics)
        return metrics

In [ ]:
class CFG:
    batch_size = 4096
    learning_rate = 1e-4
    weight_decay = 1e-4
    
    
dm = WSDMDataModule(data)
WSDMmpmodel = WSDMModule(
    features = dm,
    learning_rate=CFG.learning_rate,
    weight_decay = CFG.weight_decay,
)

trainer = pl.Trainer(
    progress_bar_refresh_rate=1,
    max_epochs=50,
    gpus=[0],
    weights_save_path='WSDM',
    logger=TensorBoardLogger("lightning_logs/", name="resnet"),
    callbacks=[LearningRateMonitor(logging_interval="step"),
               ModelCheckpoint(monitor='val_loss',dirpath='./',
                               mode = 'min',
                               filename='./output/WSDM-{epoch:02d}_{val_loss:.2f}',
                                  save_top_k = 2,
                                  save_on_train_epoch_end = True)
              ],
    amp_backend="apex",
    #accelerator='ddp'
    
)


trainer.fit(WSDMmpmodel, dm)
trainer.validate(WSDMmpmodel, dm, verbose=True)

/nfs/home/wangmingjie/.conda/envs/pytorch1.9-cuda11-闪电！/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name  | Type      | Params
------------------------------------
0 | model | WSDMModel | 13.4 K
------------------------------------
13.4 K    Trainable params
0         Non-trainable params
13.4 K    Total params
0.054     Total estimated model params size (MB)
/nfs/home/wangmingjie/.conda/en

Validation sanity check: 0it [00:00, ?it/s]

/nfs/home/wangmingjie/.conda/envs/pytorch1.9-cuda11-闪电！/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([4096])) that is different to the input size (torch.Size([4096, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/nfs/home/wangmingjie/.conda/envs/pytorch1.9-cuda11-闪电！/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1904])) that is different to the input size (torch.Size([1904, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Training: 0it [00:00, ?it/s]

/nfs/home/wangmingjie/.conda/envs/pytorch1.9-cuda11-闪电！/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([81])) that is different to the input size (torch.Size([81, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
trainer.validate(WSDMmpmodel, dm, verbose=True)